In [ ]:
# import required packages
import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn.preprocessing import scale

**Step 1: Preprocessing and Merging**

In this section I will first import the Index of Multiple Deprivation (IMD) by LSOA obtained from the MHCLG Open Data website (1) and "churning" data obtained from the CDRC website (James, 2).
Then I will preprocess them by extracting just the Leeds district data from the IMD dataset, as this contains Local Authority District names, and performing a left-hand merge with the churning data, which does not have a Local Authority District variable.
This leaves us with 482 LSOAs with data in the Leeds area 


In [ ]:
#import deprivation data
deprivation = pd.read_csv("File_1_-_IMD2019_Index_of_Multiple_Deprivation (1).csv")

#Exploring deprivation data
deprivation.head()
deprivation.tail()
deprivation.columns
deprivation.index
deprivation.shape
deprivation.dtypes
deprivation.info()
#Everything looks in order - data types are appropriate and no data is missing

In [ ]:
#First, I'm only interested in Leeds so let's subset that
leeds_deprivation = deprivation.loc[deprivation["Local Authority District name (2019)"]=="Leeds"]
#Let's check if it's worked
leeds_deprivation.head()
#Looks the same - let's use groupby to make sure only IMD is here

In [ ]:
leeds_deprivation.groupby("Local Authority District name (2019)").size()
#Looks good! 

In [ ]:
#import churning data
churning = pd.read_csv("hh_churn_lsoa11_2023.csv")

#Exploring churning data
churning.head()
churning.tail()
churning.columns
churning.index
churning.shape
churning.dtypes
churning.info()

#Again everything looks in order - data types are appropriate and no data is missing

In [ ]:
#Let's do a spatial join so we don't have to subset again
leeds_fulldata = leeds_deprivation.merge(churning, how="left", left_on="LSOA code (2011)", right_on="area")

#Did it work?
leeds_fulldata.head()
leeds_fulldata.info()
#Nothing's missing so it looks good!

**Step 2: Exploring the Data**
In this step I explore the data through multiple charts, including a histogram to check the distribution of each variable that I am interested in.
I then use hex plots, regression plots, categorical, and box plots to observe the relationships between the IMD deciles and churning variables.

In [ ]:
#Histogram to check IMD distribution
leeds_fulldata["Index of Multiple Deprivation (IMD) Decile"].hist();
#Interesting - it's relatively evenly distributed, but the number of least deprived areas is skewed much higher

In [ ]:
#Histogram to check churning distribution
#let's use percentage change between 1997 to 2023 as an example
#And break it into 20 bins since it's percentage data
leeds_fulldata["chn1997"].hist(bins=20);
#Pretty normal distribution, with a median around 65%

In [ ]:
#How does it look between 2017 and 2023?
leeds_fulldata["chn2017"].hist(bins=20);
#As expected - still a pretty normal distribution but with lower median of 20%

In [ ]:
#Let's try plotting these against each other
sns.jointplot(x="Index of Multiple Deprivation (IMD) Decile", y="chn1997", kind="hex", data=leeds_fulldata);
#Difficult to tell much of a correlation here 

In [ ]:
#Let's try a regression plot
sns.jointplot(x="Index of Multiple Deprivation (IMD) Decile", y="chn1997", kind="reg", data=leeds_fulldata);
#Looks like a relatively weak negative correlation between 1997-2023 churn and IMD decile

In [ ]:
#Let's try a categorical plot
sns.catplot(x="Index of Multiple Deprivation (IMD) Decile", y="chn1997", data=leeds_fulldata, height=6, dodge=True);

In [ ]:
#Perhaps a box plot would be best to illustrate this
sns.catplot(x="Index of Multiple Deprivation (IMD) Decile", y="chn1997", data=leeds_fulldata, height=6, dodge=True, kind="box")

**Step 3: Non-Spatial Final Visualisation**
This final visualisation is designed for Leeds City Council policymakers aiming to understand if there is a relationship between residential mobility and IMD decile. 
A box plot is the best choice for this as the categorical nature of IMD deciles is best communicated via this method. I have also used informative captions about what IMD decile means in order to inform policymakers who may be unfamiliar with IMD deciles

In [ ]:
#This seems like the best way to plot this data, so let's clean up this plot by adding a title and informative axis names
sns.catplot(x="Index of Multiple Deprivation (IMD) Decile", y="chn1997", 
                 data=leeds_fulldata, 
                 height=6, 
                 dodge=True, 
                 kind="box");

#Adding axis names and title and setting appropriate font size
plt.xlabel("2019 Index of Multiple Deprivation (IMD) Decile (1 = Most Deprived 10%; 10 = Least Deprived 10%)", fontsize = 8);
plt.ylabel("Proportion of properties which changed occupants between 1997 and 2023", fontsize = 10);
plt.title("Deprivation and Residential Changes in Leeds, By Lower Super Output Area")

**Step 4: Processing Spatial Data**
In order to present this non-spatial data in a spatial context, I will first import LSOA centroid data from the ONS (3) and merge it with the non-spatial dataset using the LSOA codes. I then convert the X and Y coordinates of the centroids into geometry (with CRS as British National Grid, the most appropriate CRS for UK data) so that they can be read and geopandas and mapped. This is my first spatial dataset.

After this, I read in an LSOA boundary shapefile from the ONS (4), set its coordinates system to WGS84 and then convert it to British National Grid. Then I merge this with the full data to create my second spatial dataset: LSOA boundaries. After this I remove any data with null values so only Leeds district LSOAs are present in the data as these are the only LSOAs I'm interested in.

The reason that I want a centroid dataset and a boundary dataset is so I can map both the churning and IMD variables on the same spatial map.

In [ ]:
#Now let's read in the shapefile of LSOA centroids
LSOACentroids = gpd.read_file("infuse_lsoa_lyr_2011.csv")

#And explore the data
LSOACentroids.head()
LSOACentroids.info()
#So we can do a join with the geo_code variable

In [ ]:
#Let's do the join now
leeds_fulldata = leeds_fulldata.merge(LSOACentroids, how="left", left_on="LSOA code (2011)", right_on="geo_code")

#Let's check it worked
leeds_fulldata.info()
#No data missing so it's all good now

In [ ]:
#Since the points are coordinates, we'll need to convert into geometry, with CRS as British National Grid
leeds_fulldata = gpd.GeoDataFrame(leeds_fulldata, geometry=gpd.points_from_xy(leeds_fulldata.x, leeds_fulldata.y), crs="EPSG:27700")

#Let's check if geometry has been added
leeds_fulldata.head()
#Geometry is there!

In [ ]:
#Let's see how it's mapped
leeds_fulldata.explore()
#Looks correct! - but they're centroids. I'd like boundaries too

In [ ]:
#Now let's read in the LSOA boundaries
LSOABoundaries = gpd.read_file("Lower_Layer_Super_Output_Areas_December_2011_Generalised_Clipped__Boundaries_in_England_and_Wales.shp")

#And explore the data
LSOABoundaries.head()
#There's no geocode here - so we'll have to do a spatial join

In [ ]:
#First let's check the CRS
LSOABoundaries.crs
#No CRS, but I know that it's WGS from the readme. So let's first set that
LSOABoundaries = LSOABoundaries.set_crs("EPSG:4326")
#And let's check it worked
LSOABoundaries.explore()
#It worked!

In [ ]:
#Now to convert it to BNG
LSOABoundaries = LSOABoundaries.to_crs("EPSG:27700")
#And let's check it worked
LSOABoundaries.crs
#And let's explore just to check!
LSOABoundaries.explore()
#Looks good. Now for the spatial join

In [ ]:
leeds_fulldata_boundaries = gpd.sjoin(leeds_fulldata, LSOABoundaries, how="right", predicate="intersects")
#Did it work?
leeds_fulldata_boundaries.info()
leeds_fulldata_boundaries.explore()
#Looks good, but LSOAs from all over England and Wales are still there
#So let's remove any rows with NaN values

In [ ]:
leeds_fulldata_boundaries = leeds_fulldata_boundaries.dropna()
#Did it work?


In [ ]:
leeds_fulldata_boundaries.info()
#Only 482 rows so that works! Now we're ready for the analysis

**Step 5: Exploring the Spatial Data**

First I plot the IMD deciles and churning deciles spatially in order to see their spatial trends.

Then I use Spearman's rank correlation to analyse the relationship between IMD decile and churning trends between 1997 and 2023, 2007 and 2023, and 2017 and 2023, and plot this in a correlation matrix. I use Spearman's rank as this is most appropriate for the not normally distributed IMD data. I use these three different churning variables as they will track how the relationship between IMD decile and churning changes depending on how "fast" residential churn takes place.

In [ ]:
#Now let's do the analysis - first let's plot the deciles
fig, ax = plt.subplots(1, 1, figsize=(10,10))
leeds_fulldata_boundaries.plot(column="Index of Multiple Deprivation (IMD) Decile",
                               linewidth=0.1,
                               categorical=True,
                               legend=True,
                               #Red blue colour map
                               cmap="RdBu",
                               ax=ax,
                               #Setting the legend location
                               legend_kwds={'loc': 'center left','bbox_to_anchor':(1,0.5)});
#Setting title
plt.title("IMD Deciles in Leeds, 2023");
#Turning off axis
plt.axis("off");

In [ ]:
#Now let's plot the churning
fig, ax = plt.subplots(1, 1, figsize=(10,10))
leeds_fulldata_boundaries.plot(column="chn1997",
                               linewidth=0.1,
                               legend=True,
                               #Red blue colour map
                               cmap="RdBu",
                               ax=ax);
#Setting title
plt.title("Churning in Leeds between 1997 and 2023");
#Turning off axis
plt.axis("off");

In [ ]:
#Let's see what correlations we can come up with between IMD and three churn datasets: 1997, 2007, 2017
#We'll use Spearman's Rank as much of the data is not normally distributed
corr_variables = leeds_fulldata_boundaries[["chn1997", "chn2007", "chn2017", 
                                            "Index of Multiple Deprivation (IMD) Decile"]].corr(method="spearman")
corr_variables
#So correlation appears consistently, moderately negative between IMD and chn1997/chn2007
#But pretty much no correlation between IMD and chn2017

In [ ]:
#Let's plot this as a heatmap to make it easier
#First let's make the mask to hide half the plot
mask = np.triu(np.ones_like(corr_variables))

#Now let's make the plot
fig,ax = plt.subplots(figsize=(8,8))
sns.heatmap(corr_variables,
            annot=True,
            cmap="RdBu",
            vmin=-1,
            vmax=1,
            mask=mask,
            linewidths=2,
            cbar_kws={"label":"Spearman's Rank correlation", "orientation": "horizontal"},
            ax=ax)

**Step 6: Cluster analysis**

In an attempt to integrate this data into clusters, I use cluster analysis. First I use the elbow method to determine the optimum number of clusters, which is three. I then plot these clusters spatially. After using the group_by command to assess the average values of each cluster, it becomes apparent that the main variable influencing each cluster is the IMD decile, while the average values for churning don't change much between clusters. Though this indicates not much of a correlation between the two variables, it does provide a useful shorthand with which to group the IMD data into three clusters of low, medium, and high deprivation, as opposed to using all 10 deciles.

In [ ]:
#Now let's do a cluster analysis using the elbow method
Sum_of_squared_distances = []

K = range(1,15)

for i in K:
    km = cluster.KMeans(n_clusters=i, init="random", random_state=123)
    km = km.fit(leeds_fulldata_boundaries[["chn1997", "chn2007", "chn2017",
    "Index of Multiple Deprivation (IMD) Decile"]].values)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, "bx-")
plt.xlabel("k")
plt.ylabel("Sum of squared distances")
plt.title("Elbow Method")
plt.show
#Looks like three clusters is optimal

In [ ]:
km3 = cluster.KMeans(n_clusters=3, init="random", random_state=123)
km3cls = km3.fit(leeds_fulldata_boundaries[["chn1997", "chn2007", "chn2017",
                                            "Index of Multiple Deprivation (IMD) Decile"]].values)
#Now let's look at the labels
km3cls.labels_

In [ ]:
#Now let's add cluster as a label
leeds_fulldata_boundaries["Cluster"] = km3cls.labels_
leeds_fulldata_boundaries["Cluster"].head()

In [ ]:
#Let's see how these clusters look spatially
f, ax = plt.subplots(1, figsize=(10,10))
leeds_fulldata_boundaries.plot(column="Cluster", categorical=True, legend=True,
                               linewidth=0.1, edgecolor="white", ax=ax)

ax.set_axis_off()
plt.show()

In [ ]:
#I'll now use groupby to assess the clusters - median is best here due to the IMD's categorical nature
leeds_fulldata_boundaries_mean=leeds_fulldata_boundaries.groupby("Cluster")[["chn1997", "chn2007",
                                                                               "chn2017", "Index of Multiple Deprivation (IMD) Decile"]].mean().reset_index()
leeds_fulldata_boundaries_mean
#So the main differentiating factor here seems to be IMD decile
#This isn't really helpful for assessing multidimensional clusters
#But it's a handy way to group IMD deciles into three main groups
#0 is low mean deprivation, 1 is medium mean deprivation, 2 is high mean deprivation

In [ ]:
#So let's put these clusters and their descriptions into the dataset
leeds_fulldata_boundaries.loc[leeds_fulldata_boundaries["Cluster"]==0, "Cluster_description"]="Low average deprivation in 2019"
leeds_fulldata_boundaries.loc[leeds_fulldata_boundaries["Cluster"]==1, "Cluster_description"]="Medium average deprivation in 2019"
leeds_fulldata_boundaries.loc[leeds_fulldata_boundaries["Cluster"]==2, "Cluster_description"]="High average deprivation in 2019"

**Step 7: Finalised Spatial Data Visualiation**

Again, this visualisation is aimed towards Leeds City Council policymakers seeking to understand the relationship between churn and deprivation. I use the IMD decile clusters plotted to the boundaries as this is categorical data with fewer variables and therefore fewer colours, so is better as a background colour as it makes the visualisation look less busy and easier to interpret. I then mapped the churn data to the centroids and used a continuous blue colour scale which shows up well on the copper colours of the categorical data. Finally I used a loop function to map the three churn variables in three subplots in order to illustrate changes over time. I also ensured that the colour maps I used did not have overlapping colours so that it is clear which colour corresponds to which variable.

In [ ]:
#And now let's map the clusters and the churning data together, comprised of three subplots of different churn years

#First let's make a list of the three churn variables we want
churn_cols = ["chn1997", "chn2007", "chn2017"]

#Then we'll use a loop to save writing the same code for three churning variables
for i in range (0, len(churn_cols)):

    #This line sets size of the subplots
    fig, ax = plt.subplots(1, 1, figsize=(15,15
                                         ))
    #This sets the basemap as the clusters
    base = leeds_fulldata_boundaries.plot(ax=ax, column="Cluster_description",
                                          #The copper colour scheme is an appropriate continuous colour scheme for a base map
                                          cmap="copper",
                                          #Make the LSOA boundary lines wider
                                          linewidth=1,
                                          #White shows up better than black as a line dividers
                                          edgecolor="white",
                                          categorical=True,
                                          legend=True
                                         )
    #Then map the churn data as centroids on top of the base layer, using the "blues" colour map which shows up well
    leeds_fulldata.plot(column=churn_cols[i], ax=base, cmap="Blues", legend=True, s=20)
    #Turn the axis off
    plt.axis("off")
    plt.title(churn_cols[i].replace("chn","Proportion of properties which changed occupants between ")+" and 2023 in Leeds, with deprivation data")
    plt.show()

**Reference List**

1. MHCLG. 2019. English Indices of Deprivation 2019 - LSOA Level. *MHCLG* [Online]. [Accessed 10 May 2025]. Available from: https://opendatacommunities.org/data/societal-wellbeing/imd2019/indices

2. James, T. 2024. CDRC Residential Mobility Index LSOA 1997-2023. *CDRC* [Online]. [Accessed 10 May 2025]. Available from: https://data.cdrc.ac.uk/dataset/cdrc-residential-mobility-index

3. ONS. 2024. Lower layer Super Output Areas (December 2011) EW Population Weighted Centroids. *ONS* [Online]. [Accessed 10 May 2025]. Available from: https://www.data.gov.uk/dataset/5ac2aae6-4f82-4d71-92d6-95e648b085e1/lower-layer-super-output-areas-december-2011-ew-population-weighted-centroids

4. ONS. 2016. Lower Layer Super Output Area (LSOA) boundaries. *ONS* [Online]. [Accessed 10 May 2025]. Available from: https://www.data.gov.uk/dataset/fa883558-22fb-4a1a-8529-cffdee47d500/lower_layer_super_output_area_lsoa_boundaries 